# 1. Preprocessing

## Thach thuc

- Tin hieu EEG co nhieu cao (artifact, EMG noise)
- SNR thap → Accuracy giam 10-15%

## Y tuong

**Chebyshev Type II filter** thay vi Butterworth

| Filter | Transition Width | SNR |
|--------|-----------------|-----|
| Butterworth | Rong | Baseline |
| Chebyshev II | Hep | **+2.5 dB** |

**Ket qua:** Giu duoc nhieu tin hieu mu/beta (8-30 Hz) hon

## Code minh hoa

In [1]:
import mne

# Load data
raw = mne.io.read_raw_gdf('./data/A01T.gdf', preload=True, verbose=False)

# Pipeline: Channel selection → Filter → CAR → Epochs
raw.pick_types(eeg=True, exclude=['EOG'])
raw.filter(8, 30, method='iir', 
           iir_params={'order': 5, 'ftype': 'cheby2', 'rs': 40}, verbose=False)
raw.set_eeg_reference('average', verbose=False)

# Extract epochs (0.5-2.5s, Left vs Right hand only)
events, event_dict = mne.events_from_annotations(raw, verbose=False)
event_id = {'left_hand': 7, 'right_hand': 8}  # Event IDs 7, 8 tu file
epochs = mne.Epochs(raw, events, event_id=event_id, tmin=0.5, tmax=2.5, 
                    baseline=None, preload=True, event_repeated='drop', verbose=False)

print(f"Output shape: {epochs.get_data().shape}")
print(f"Left: {len(epochs['left_hand'])}, Right: {len(epochs['right_hand'])}")

c:\Users\ducmi\AppData\Local\Programs\Python\Python39\lib\contextlib.py:126: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Output shape: (144, 25, 501)
Left: 72, Right: 72
